
# Genome-Wide Association Study (GWAS) Coursework Demo

This notebook demonstrates a complete mini-GWAS pipeline using **PLINK** for association testing and **Python** for visualization.

**Objectives:**
- Load and inspect synthetic genotype/phenotype data.
- Run a basic case/control GWAS using PLINK.
- Visualize association results using a Manhattan plot.

**Dataset details:**
- 200 individuals, 1,000 SNPs across 22 chromosomes.
- Contains a few embedded "causal" SNPs that should produce visible peaks.
- Files included in this coursework bundle:
  - `dataset.ped`, `dataset.map`
  - `metadata.json`
  - `plot_manhattan.py`


## 1. Inspect the dataset

In [ ]:

import pandas as pd, json

# Load metadata
with open('metadata.json') as f:
    meta = json.load(f)

print("Number of individuals:", meta['n_ind'])
print("Number of SNPs:", meta['n_snp'])
print("Cases:", meta['n_cases'], "Controls:", meta['n_controls'])
print("Causal SNPs:", meta['causal_snps'])

# Display first few lines of the MAP file
pd.read_csv('dataset.map', sep='\t', header=None, names=['CHR','SNP','CM','BP']).head()


## 2. Run PLINK association test


To perform a basic allelic association test, run the following command **in a terminal** (not inside this notebook):
```bash
plink --file dataset --assoc --out results
```
This will produce `results.assoc` with p-values for each SNP.

The relevant columns are:
- `CHR` — Chromosome number  
- `SNP` — SNP identifier  
- `BP` — Base-pair position  
- `P` — p-value for association


## 3. Visualize results: Manhattan plot

In [ ]:

import matplotlib.pyplot as plt
import numpy as np

# Read association file (assuming PLINK has been run)
assoc = pd.read_csv('results.assoc', delim_whitespace=True)

assoc = assoc.dropna(subset=['P'])
assoc['-log10P'] = -np.log10(assoc['P'].replace(0, 1e-300))
assoc = assoc.sort_values(['CHR','BP'])
assoc['ind'] = range(len(assoc))

plt.figure(figsize=(12,6))
for c in sorted(assoc['CHR'].unique()):
    d = assoc[assoc['CHR']==c]
    plt.scatter(d['ind'], d['-log10P'], s=10)
bonf = 0.05 / len(assoc)
plt.axhline(-np.log10(bonf), color='red', linestyle='--', linewidth=1)
plt.xlabel('Chromosome')
plt.ylabel('-log10(p)')
plt.title('Manhattan plot (PLINK results)')
plt.tight_layout()
plt.show()


## 4. Interpretation


- Peaks above the red dashed line (Bonferroni threshold) indicate SNPs significantly associated with the phenotype.  
- Check whether these correspond to the known causal SNPs from `metadata.json` (e.g. `rs100`, `rs400`, `rs700`).
- Consider how false positives or small sample size may affect the signal.

**Extension ideas:**
- Add covariates (e.g. sex) with `--covar` option in PLINK.
- Use QQ plot to assess p-value inflation.
- Try modifying allele frequencies or sample sizes to explore statistical power.
